In [71]:
// dimensiones arena 
let anchoArena = 700.0
let altoArena = 1200.0

// distribución de los bloques
let filasBloques = 8
let columnasBloques = 14

// anchura de cada bloque: 700/14 = 50, los 14 bloques ocupan todo el ancho de la arena
let anchoBloque = 50.0

// altura de cada bloque: (1200/8)/6 = 25, los 8 bloques ocupan una sexta parte de la altura de la arena
let altoBloque = 25.0

// los bloques eatan entre la altura 800 y la altura 1000; y = [800, 1000)
let yInicial = 800.0
let yFinal = 1000.0

In [72]:
// creamos la matriz de bloques
let generarBloques (filas: int) (columnas: int) : bool[,] =
    Array2D.create filas columnas true

In [73]:
let testBloques = generarBloques filasBloques columnasBloques
printfn "Bloques: %A" testBloques

Bloques: [[true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]]


In [74]:
// funcionnes para obtener los indices de un bloque a partir de un par de coordenadas (x,y)

// verifica que las coordenadas esten dentro de la región donde existen bloques
let validarCoordenadas (x: float) (y: float): bool =
    x >= 0.0 && x < anchoArena && y >= yInicial && y < yFinal

// obtiene los indices de un bloque a partir de un par de coordenadas (x,y)
let obtenerIdxBloque (x: float) (y: float): int*int =
    let indiceFila = int ((y - yInicial) / altoBloque)
    let indiceColumna = int (x / anchoBloque)
    (indiceFila, indiceColumna)


let testCoordenadas (x: float) (y: float): unit =
    match validarCoordenadas x y with
    | false -> printfn "No hay bloques en estas coordenadas"
    | true ->
        let (fila, columna) = obtenerIdxBloque x y
        if fila >= 0 && fila < filasBloques && columna >= 0 && columna < columnasBloques then
            printfn $"Las coordenadas corresponden al bloque ({fila}, {columna})"
        else
            printfn "Los indices obtenidos estan fuera de rango"      

In [75]:
testCoordenadas 0.0 800.0
testCoordenadas 0.0 1000.0
testCoordenadas 0.0 999.999
testCoordenadas 700.0 999.999
testCoordenadas 699.999 800.0
testCoordenadas 699.999 1000.0
testCoordenadas 699.999 999.999

Las coordenadas corresponden al bloque (0, 0)
No hay bloques en estas coordenadas
Las coordenadas corresponden al bloque (7, 0)
No hay bloques en estas coordenadas
Las coordenadas corresponden al bloque (0, 13)
No hay bloques en estas coordenadas
Las coordenadas corresponden al bloque (7, 13)


In [76]:
// funciones para el evento de colisión con un bloque

// verifica si hay un bloque en las coordenadas (x,y)
let hayBloque (x: float) (y: float) (bloques: bool[,]) : bool =
    match validarCoordenadas x y with
    | false -> false
    | true ->
        let (indiceFila, indiceColumna) = obtenerIdxBloque x y
        bloques.[indiceFila, indiceColumna]


let testHayBloque (x: float) (y: float) (bloques: bool[,]) : unit =
    match hayBloque x y bloques with
    | true -> printfn "Hay un bloque en las coordenadas x=%f, y=%f" x y
    | false -> printfn "No hay un bloque en las coordenadas x=%f, y=%f" x y


// cambia el estado de un bloque a false
let desactivarBloque (x: int) (y: int) (bloques: bool[,]) : unit =
    bloques.[x, y] <- false


let colisionConBloque (x: float) (y: float) (bloques: bool[,]) : unit =
    match hayBloque x y bloques with
    | false -> ()
    | true ->
        let (indiceFila, indiceColumna) = obtenerIdxBloque x y
        desactivarBloque indiceFila indiceColumna bloques           

In [77]:
let testX = 0.0
let testY = 800.0
let testBloques: bool[,] = Array2D.create filasBloques columnasBloques true
let (idxFila, idxColumna) = obtenerIdxBloque testX testY

printfn $"Las coordenadas x={testX}, y={testY} corresponden al bloque ({idxFila}, {idxColumna})"
printfn "Bloques antes de la colision:"
printfn "%A" testBloques
testHayBloque testX testY testBloques
colisionConBloque testX testY testBloques
printfn "Bloques despues de la colision:"
printfn "%A" testBloques
testHayBloque testX testY testBloques

Las coordenadas x=0, y=800 corresponden al bloque (0, 0)
Bloques antes de la colision:
[[true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; true; true; true; true; true; true; true; true; true;
  true]]
Hay un bloque en las coordenadas x=0.000000, y=800.000000
Bloques despues de la colision:
[[false; true; true; true; true; true; true; true; true; true; true; true; true;
  true]
 [true; true; true; true; tru